In [1]:
!pip install xgboost


  Using cached xgboost-1.7.6-py3-none-win_amd64.whl (70.9 MB)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
# Load the dataset
data = pd.read_csv("C:\\Users\\vijaya bhavani\\Downloads\\churn_dataset.csv")

In [5]:
# Data cleaning: Handling missing values
data.fillna(method="ffill", inplace=True) 

In [6]:
# Feature engineering: Creating new features
data["total_products_balance"] = data["NumOfProducts"] * data["Balance"]
data["credit_to_salary_ratio"] = data["CreditScore"] / data["EstimatedSalary"]


In [7]:
# Separate categorical and numerical features
categorical_features = ["Geography", "Gender"]
numerical_features = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]

In [8]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), categorical_features)
    ])

In [9]:
# Apply preprocessing pipeline
X = data.drop("Exited", axis=1) 
y = data["Exited"]


In [10]:
X_preprocessed = preprocessor.fit_transform(X)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [12]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(X_train_scaled, y_train)
logreg_preds = logreg_model.predict(X_test_scaled)


In [14]:
# Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
rf_preds = rf_model.predict(X_test_scaled)

In [15]:
# Gradient Boosting model (XGBoost)
xgb_model = XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)
xgb_preds = xgb_model.predict(X_test_scaled)


In [16]:
# Evaluate the models
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    print(f"=== {model_name} ===")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:\n", report)


In [17]:
evaluate_model(y_test, logreg_preds, "Logistic Regression")
evaluate_model(y_test, rf_preds, "Random Forest")
evaluate_model(y_test, xgb_preds, "XGBoost")

=== Logistic Regression ===
Accuracy: 0.8110
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89      1607
           1       0.55      0.20      0.29       393

    accuracy                           0.81      2000
   macro avg       0.69      0.58      0.59      2000
weighted avg       0.78      0.81      0.77      2000

=== Random Forest ===
Accuracy: 0.8660
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.76      0.47      0.58       393

    accuracy                           0.87      2000
   macro avg       0.82      0.72      0.75      2000
weighted avg       0.86      0.87      0.85      2000

=== XGBoost ===
Accuracy: 0.8610
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      1607
           1       0.70      0.51      0.59     